# Quiz 1

In [3]:
import numpy as np
import cv2

In [2]:
# Q1
def point_line_distance(line, p):
    """
    Calculate shortest distance d between line l
    and 2D homogenous point p.

    e.g.
    line = np.array([1,0,1])
    p = np.array([0,0,1])
    """
    d = abs(line @ p) / (abs(p[2]) * np.sqrt(line[0] ** 2 + line[1] ** 2))
    return d


q = np.array([2, 4, 3]).T
line = np.array([1, 2, 2]).T
point_line_distance(line, q)

2.3851391759997758

In [3]:
# Q2
def Pi(ph):
    """
    Converts coordinates from homogeneous to inhomogeneous.
    ph : 4xn np.array
    p : 3xn np.array
    """
    p = ph[:-1] / ph[-1]  # divide by and remove last coordinate
    return p


def PiInv(p):
    """
    Converts coordinates from inhomogeneous to homogeneous.
    p : 3xn np.array
    ph : 4xn np.array
    """
    ph = np.vstack((p, np.ones(p.shape[1])))
    return ph


# pose
R = cv2.Rodrigues(np.array([-0.1, 0.1, -0.2]))[0]
t = np.array([[0.09], [0.05], [0.05]])

Q = np.array([-0.03, 0.01, 0.59])
Q = Q.reshape(-1, 1)
print("Q:", Q.shape)

# camera intrinsics
f = 1720
pp = (680, 610.0)
K = np.array([[f, 0, pp[0]], [0, f, pp[1]], [0, 0, 1]])

# projection onto image plane
Qh = PiInv(Q)  # 4 x 1
print(Qh)
pose = np.hstack((R, t))  # 3 x 4
print(pose)
Ph = K @ pose @ Qh  # 3 x 1
print(Ph)
P = Pi(Ph)  # 2 x 1
P

Q: (3, 1)
[[-0.03]
 [ 0.01]
 [ 0.59]
 [ 1.  ]]
[[ 0.97512475  0.19303094  0.1089531   0.09      ]
 [-0.20298104  0.97512475  0.0890529   0.05      ]
 [-0.0890529  -0.1089531   0.9900499   0.05      ]]
[[6.50653114e+02]
 [5.91400859e+02]
 [6.35711497e-01]]


array([[1023.50377104],
       [ 930.29756751]])

In [4]:
# from Copilot

import numpy as np


def project_3d_point_to_2d(
    point_3d, focal_length, principal_point, rotation_matrix, translation_vector
):
    # Create the camera intrinsic matrix
    intrinsic_matrix = np.array(
        [
            [focal_length, 0, principal_point[0]],
            [0, focal_length, principal_point[1]],
            [0, 0, 1],
        ]
    )

    # Create the camera extrinsic matrix
    extrinsic_matrix = np.hstack((rotation_matrix, translation_vector))

    # Create the camera projection matrix
    projection_matrix = np.dot(intrinsic_matrix, extrinsic_matrix)

    # Append a 1 to the 3D point to create a homogeneous coordinate
    point_3d_homogeneous = np.append(point_3d, 1)

    # Project the 3D point to 2D
    point_2d_homogeneous = np.dot(projection_matrix, point_3d_homogeneous)

    # Convert the 2D point from homogeneous to Cartesian coordinates
    point_2d = point_2d_homogeneous[:2] / point_2d_homogeneous[2]

    return point_2d


ans = project_3d_point_to_2d(Q, f, pp, R, t)
ans

array([1023.50377104,  930.29756751])

# Quiz 2

In [5]:
# Question 2
def camera_intrinsic(f, c, alpha=1, beta=0):
    """
    Create a camera intrinsic matrix

    f : float, focal length
    c : 2D principal point
    alpha : float, skew
    beta : float, aspect ratio
    """
    K = np.array([[f, beta * f, c[0]], [0, alpha * f, c[1]], [0, 0, 1]])
    return K

In [6]:
K = camera_intrinsic(1200, (400, 350), 1, 0)
K
# radial distortion should not affect the camera (intrinsic?) matrix

array([[1200,    0,  400],
       [   0, 1200,  350],
       [   0,    0,    1]])

In [7]:
K = np.array([[1000, 0, 400], [0, 1000, 300], [0, 0, 1]])
q = np.array([1, 1, 1])

# Quiz 3

In [8]:
import numpy as np
import cv2

from util_functions import (
    fundamental_matrix,
    point_line_distance,
    projection_matrix,
    triangulate,
    PiInv,
)

In [9]:
# Copied from question
K = np.array([[900, 0, 1070], [0, 900, 610.0], [0, 0, 1]], float)
R1 = cv2.Rodrigues(np.array([-1.6, 0.3, -2.1]))[0]
t1 = np.array([[0.0], [1.0], [3.0]], float)
R2 = cv2.Rodrigues(np.array([-0.4, -1.3, -1.6]))[0]
t2 = np.array([[0.0], [1.0], [6.0]], float)
R3 = cv2.Rodrigues(np.array([2.5, 1.7, -0.4]))[0]
t3 = np.array([[2.0], [-7.0], [25.0]], float)

# Observed points with noise
p1 = np.array([[1046.0], [453.0]])
p2 = np.array([[1126.0], [671.0]])
p3 = np.array([[1165.0], [453.0]])

In [10]:
F = fundamental_matrix(K, R1, t1, K, R2, t2)

# Epipolar line of p1 in camera 2
l2 = F @ PiInv(p1)

# Calculate distance of p2 to epipolar line
d = point_line_distance(l2, PiInv(p2))
d

array([[13.27182907]])

In [11]:
q_list = [p1, p2, p3]

P1 = projection_matrix(K, R1, t1)
P2 = projection_matrix(K, R2, t2)
P3 = projection_matrix(K, R3, t3)
P_list = [P1, P2, P3]

# Triangulate the points
Q = triangulate(q_list, P_list)
Q

array([[3.10058867],
       [0.74321098],
       [0.46490561]])

# Quiz 6

In [12]:
data = np.load("media/harris.npy", allow_pickle=True).item()
print(data.keys())

gxx = data["g*(I_x^2)"]
gyy = data["g*(I_y^2)"]
gxy = data["g*(I_x I_y)"]

k = 0.06
tau = 516

dict_keys(['g*(I_x^2)', 'g*(I_y^2)', 'g*(I_x I_y)'])


In [13]:
# Structure tensor
C00 = gxx
C01 = gxy
C10 = C01
C11 = gyy
C = np.array([[C00, C01], [C10, C11]])

# Harris measure
a = C[0, 0]
b = C[1, 1]
c = C[0, 1]
r = a * b - c**2 - k * (a + b) ** 2

In [14]:
# Perform 4-neigbourhood non-max suppression
c = []
for i in range(1, gxx.shape[0] - 1):
    for j in range(1, gxx.shape[1] - 1):
        if (
            r[i, j] > r[i + 1, j]
            and r[i, j] >= r[i - 1, j]
            and r[i, j] > r[i, j + 1]
            and r[i, j] >= r[i, j - 1]
            and r[i, j] > tau
        ):  # Threshold
            c.append([i, j])

print(f"Corners found at {c}")

Corners found at [[2, 1]]


# Quiz 7

In [15]:
data = np.load("media/ransac.npy", allow_pickle=True).item()
data

{'points': array([[0.06911248, 0.60836222, 0.07749226, 0.3808085 , 0.39192472,
         0.64590358, 0.48075454, 0.85126341, 0.98751457, 0.1585098 ,
         0.42800112, 0.35318965, 0.81344836, 0.02267745, 0.54530502,
         0.93877605, 0.05661325, 0.22504892, 0.1182399 , 0.09128742,
         0.4009394 , 0.27185581, 0.41392803, 0.5737753 , 0.27623313,
         0.0824709 , 0.57587571, 0.5780767 , 0.96663698, 0.38222185,
         0.32846916, 0.83400574, 0.58624363, 0.23966859, 0.98910107,
         0.88695829, 0.40939304, 0.78976526, 0.19805607, 0.81784108,
         0.34157588, 0.05209379, 0.31898118, 0.21285602, 0.00789091,
         0.49359988, 0.84910533, 0.89311695, 0.52778303, 0.83260772,
         0.37146218, 0.15683103, 0.51521828, 0.89553227, 0.27440068,
         0.40155061, 0.15598586, 0.59476958, 0.58388262, 0.07273128,
         0.76430047, 0.57452332, 0.80745292, 0.41690103, 0.9622128 ,
         0.63636018, 0.41402553, 0.98782043, 0.73864474, 0.83948487,
         0.77731663, 0.9

In [16]:
def fit_line(p1, p2):
    """
    Fits a line given 2 points.

    Args:
        p1, p2 (np.array) : 2x1 inhomogenous coordinates

    Returns:
        l : 3x1, line in homogenous coordinates
    """
    if p1.shape == (2,):
        p1 = p1.reshape(2, 1)
        p2 = p2.reshape(2, 1)
    if p1.shape != (2, 1) or p2.shape != (2, 1):
        raise ValueError("Points must be 2x1 np.array")

    p1h = PiInv(p1)
    p2h = PiInv(p2)
    # cross() requires input as vectors
    l = np.cross(p1h.squeeze(), p2h.squeeze())
    return l


def find_inliners_outliers(l, points, tau):
    """
    Args:
        l : equation of line in homogenous coordinates
        tau : threshold for inliners

    Returns:
        inliners (np.array) : 2xa, set of inliner points
        outliers (np.array) : 2xb, set of outlier points
    """
    inliners = []
    outliers = []
    for p in points.T:
        p = p.reshape(2, 1)
        ph = PiInv(p)
        d = abs(l.T @ ph) / (abs(ph[2]) * np.sqrt(l[0] ** 2 + l[1] ** 2))
        if d <= tau:  # inliner
            inliners.append(p)
        else:  # outlier
            outliers.append(p)
    inliners = np.array(inliners).squeeze().T
    outliers = np.array(outliers).squeeze().T
    return inliners, outliers


# Load data
points = data["points"]
x1 = data["x1"]
x2 = data["x2"]
tau = 0.2

# Fit line
l = fit_line(x1, x2)
print(f"line eqn: {l}")

inliners, outliers = find_inliners_outliers(l, points, tau)
print(f"inliners: {inliners.shape}")

line eqn: [ 0.13825614  0.10280998 -0.16058676]
inliners: (2, 34)


In [19]:
best_consensus = 103
p = 0.95
n_points = 404
# 8 DoF in homography matrix
# but, only 4 pairs of points are needed to estimate homography
n = 4

# Stopping criteria
e_est = 1 - best_consensus / n_points
N_est = np.log(1 - p) / np.log(1 - (1 - e_est) ** n)
N_est

707.5554825030302

# Quiz 10

In [8]:
chi_sq = 4.61  # 90% confidence
sigma = 2.6
threshold = (chi_sq * sigma**2) ** 0.5
threshold

1.1914696806885186